In [1]:
import pandas as pd
import geopandas as gpd
from siuba import *

# from segment_speed_utils import gtfs_schedule_wrangling, helpers, segment_calcs,sched_rt_utils
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    PROJECT_CRS,
    SEGMENT_GCS,
)
import datetime as dt

In [2]:
import shared_utils

In [3]:
from shared_utils.rt_utils import MPH_PER_MPS

In [4]:
target_mph = 16
target_mps = target_mph / MPH_PER_MPS

In [5]:
SEGMENT_GCS

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/'

# Lincoln Corridor Speed History

* support D7 Lincoln Bus Lane Feasibility Study by illustrating worsening speeds and bus rider delay over the last few years
* will refactor into a script pending speedmap segments work (goal is to align on those)

In [6]:
# analysis_date = shared_utils.rt_dates.DATES['mar2023']
analysis_date = shared_utils.rt_dates.DATES['sep2022a']

In [7]:
#  from Oct 12 22 RtFilterMapper run, handy to use as a clip geom too
corr = gpd.read_parquet('../rt_delay/lincoln_oct12_old.parquet')

In [14]:
feeds = shared_utils.gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(analysis_date)

In [16]:
def get_bbb_feed(analysis_date):

    feeds = shared_utils.gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(analysis_date)
    bbb_feed = feeds >> filter(_.name.str.contains('Big Blue'))
    return bbb_feed

# bbb_dataset_key = bbb_feed.gtfs_dataset_key.iloc[0]
# bbb_feed_key = bbb_feed.feed_key.iloc[0]

In [17]:
# mar23 = pd.read_parquet(f'{SEGMENT_GCS}speeds_stop_segments_{analysis_date}.parquet')

In [18]:
# mar23_shape = pd.read_parquet(f'{SEGMENT_GCS}segment_options/shape_stop_segments_{analysis_date}.parquet')

In [70]:
def get_shapes(analysis_date, bbb_feed):
    # shapes = pd.read_parquet(f'{SEGMENT_GCS}segment_options/shape_stop_segments_{analysis_date}.parquet')
    shapes = pd.read_parquet(f'{SEGMENT_GCS}segment_options/speedmap_segments_{analysis_date}.parquet')
    bbb_shapes = shapes >> filter(_.schedule_gtfs_dataset_key == bbb_feed.gtfs_dataset_key.iloc[0])
    bbb_shapes.geometry = gpd.GeoSeries.from_wkb(bbb_shapes.geometry)
    bbb_shapes = gpd.GeoDataFrame(bbb_shapes, geometry=bbb_shapes.geometry, crs=PROJECT_CRS)
    return bbb_shapes

In [71]:
def get_speeds(analysis_date, bbb_feed):
    # speeds = pd.read_parquet(f'{SEGMENT_GCS}speeds_stop_segments_{analysis_date}.parquet')
    speeds = pd.read_parquet(f'{SEGMENT_GCS}speedmap/speeds_{analysis_date}.parquet')
    bbb_speeds = (speeds >> filter(_.schedule_gtfs_dataset_key == bbb_feed.gtfs_dataset_key.iloc[0])
                     >> select(_.speed_mph, _.meters_elapsed, _.sec_elapsed, _.trip_instance_key, _.stop_sequence)
                 )
    return bbb_speeds

In [72]:
def lincoln_corridor_delay(analysis_date):
    
    # print(analysis_date)
    bbb_feed = get_bbb_feed(analysis_date)
    bbb_speeds = get_speeds(analysis_date, bbb_feed)
    bbb_shapes = get_shapes(analysis_date, bbb_feed)
    
    lincoln_trips = shared_utils.gtfs_utils_v2.get_trips(selected_date=analysis_date,
                                                 operator_feeds=[bbb_feed.feed_key.iloc[0]],
                                                custom_filtering = {'route_short_name': ['3', 'R3']})
    lincoln_trips = lincoln_trips >> select(_.trip_instance_key, _.route_short_name, _.trip_first_departure_ts)
    sched_trip_count = lincoln_trips.shape[0]

    lincoln_shapes = bbb_shapes >> inner_join(_, lincoln_trips, on = ['trip_instance_key'])

    lincoln_shapes = lincoln_shapes >> inner_join(_, bbb_speeds, on=['trip_instance_key', 'stop_sequence'])
    print((lincoln_shapes >> count(_.trip_instance_key)).shape)
    assert not lincoln_shapes.empty
    rt_trip_count = (lincoln_shapes >> count(_.trip_instance_key)).shape[0]
    
    lincoln_shapes['target_sec_elapsed'] = lincoln_shapes.meters_elapsed / target_mps
    assert lincoln_shapes.crs == corr.crs
    lincoln_clipped = gpd.clip(lincoln_shapes, corr)
    lincoln_clipped['speed_delay'] = (lincoln_clipped.sec_elapsed - lincoln_clipped.target_sec_elapsed).clip(lower=0)
    delay_minutes = lincoln_clipped.speed_delay.sum() / 60
    
    return (analysis_date, delay_minutes, rt_trip_count, rt_trip_count / sched_trip_count)

In [63]:
lincoln_corridor_delay(analysis_date)

/tmp/ipykernel_908/2986224776.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbb_shapes.geometry = gpd.GeoSeries.from_wkb(bbb_shapes.geometry)


(182, 2)


('2022-09-21', 1181.8942798723115, 182, 0.9285714285714286)

In [73]:
# corr.explore() #  limited to just Feasibility Study (Venice-Manchester)

In [65]:
test = list(shared_utils.rt_dates.DATES.keys())[50:]

In [66]:
test

['may2024', 'jun2024', 'jul2024']

In [67]:
def compile_lincoln_delays():
    lincoln_delays = []
    # for date in shared_utils.rt_dates.DATES.keys():
    for date in test:
        analysis_date = shared_utils.rt_dates.DATES[date]
        print(analysis_date)
        try:
            lincoln_delays += [lincoln_corridor_delay(analysis_date)]
        except:
            print(f'failed for {analysis_date}')
    # return lincoln_delays
    lincoln_delay_df = pd.DataFrame(lincoln_delays,
                                    columns=['date_str', 'delay_minutes', 'vp_trip_count',
                                             'vp_pct_sched'])
    lincoln_delay_df['date'] = lincoln_delay_df.date_str.apply(lambda x: dt.datetime.fromisoformat(x).date())
    lincoln_delay_df['weekday'] = lincoln_delay_df.date.apply(lambda x: x.strftime('%a'))
    # df = df >> filter(-_.weekday.isin(['Sat', 'Sun', 'Mon', 'Fri']))
    return lincoln_delay_df

In [68]:
lincoln_delay_df = compile_lincoln_delays()

2024-05-22


/tmp/ipykernel_908/2986224776.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbb_shapes.geometry = gpd.GeoSeries.from_wkb(bbb_shapes.geometry)


(98, 2)
2024-06-12
(0, 2)
failed for 2024-06-12
2024-07-17


/tmp/ipykernel_908/2986224776.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbb_shapes.geometry = gpd.GeoSeries.from_wkb(bbb_shapes.geometry)


(41, 2)


In [69]:
lincoln_delay_df

,date_str,delay_minutes,vp_trip_count,vp_pct_sched,date,weekday
0,2024-05-22,714.266458,98,0.604938,2024-05-22,Wed
1,2024-07-17,216.935784,41,0.253086,2024-07-17,Wed


In [33]:
lincoln_delay_df = lincoln_delay_df >> filter(_.vp_pct_sched > .8)

In [34]:
lincoln_delay_df['delay_per_trip'] = lincoln_delay_df.delay_minutes / lincoln_delay_df.vp_trip_count

In [35]:
lincoln_delay_df = lincoln_delay_df >> filter(-_.weekday.isin(['Sat', 'Sun', 'Mon', 'Fri']))

In [36]:
lincoln_delay_df

,date_str,delay_minutes,vp_trip_count,vp_pct_sched,date,weekday,delay_per_trip
0,2022-09-21,1365.976285,182,0.928571,2022-09-21,Wed,7.505364
1,2023-03-15,936.822081,156,0.962963,2023-03-15,Wed,6.005270
3,2023-04-11,1345.995844,144,0.888889,2023-04-11,Tue,9.347193
4,2023-04-12,967.829197,160,0.987654,2023-04-12,Wed,6.048932
5,2023-04-13,974.676955,146,0.901235,2023-04-13,Thu,6.675870
9,2023-05-17,1066.920026,143,0.882716,2023-05-17,Wed,7.460979
10,2023-06-14,958.144675,144,0.888889,2023-06-14,Wed,6.653782
11,2023-07-12,1096.543234,162,1.000000,2023-07-12,Wed,6.768785
12,2023-08-15,1229.532042,160,0.987654,2023-08-15,Tue,7.684575
13,2023-09-13,1068.675801,147,0.907407,2023-09-13,Wed,7.269903


In [37]:
import altair as alt

In [38]:
lincoln_delay_df.date = lincoln_delay_df.date.apply(lambda x: pd.Timestamp(x))

In [39]:
chart = alt.Chart(lincoln_delay_df).mark_point().encode(
    x=alt.X('date:T', axis = alt.Axis(title = 'Date'.upper(), format = ("%b %Y"))),
    y='delay_per_trip'
).properties(width=600, height=400)

chart + chart.transform_regression('date', 'delay_per_trip').mark_line()

alt.LayerChart(...)

In [40]:
lincoln_delay_df.to_csv('lincoln_delays_preliminary.csv')

In [41]:
(10 - 8) / 8

0.25